In [1]:
import numpy as np
import random
import pandas as pd
import gzip
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
n = 40428967  #total number of records in the clickstream data 
sample_size = 100000
skip_values = sorted(random.sample(range(1,n), n-sample_size))
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
with gzip.open('avazu/train.gz') as f:
    train = pd.read_csv(f, parse_dates = ['hour'], date_parser = parse_date, skiprows = skip_values)

In [3]:
train = train.rename(columns={'hour': 'date'})

In [4]:
#which hours have the most clicks 
train['hour'] = train.date.apply(lambda x: x.hour)
train['day_of_week'] = train['date'].apply(lambda val: val.weekday_name)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
train.head()

,id,click,date,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,C14,C15,C16,C17,C18,C19,C20,C21,hour,day_of_week
0,10009807995169380879,0,2014-10-21,1005,0,85f751fd,c4e18dd6,50e219e0,396df801,2347f47a,...,15705,320,50,1722,0,35,100084,79,0,Tuesday
1,10138529696369058965,0,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,15702,320,50,1722,0,35,-1,79,0,Tuesday
2,10165706789569778048,0,2014-10-21,1005,0,85f751fd,c4e18dd6,50e219e0,6fc85e22,7801e8d9,...,18993,320,50,2161,0,35,-1,157,0,Tuesday
3,10251257831980779397,1,2014-10-21,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,15701,320,50,1722,0,35,100084,79,0,Tuesday
4,10282973710434293213,0,2014-10-21,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,...,17614,320,50,1993,2,1063,100084,33,0,Tuesday


In [6]:
target = train['click']

In [7]:
analysis = train
analysis.drop('date', axis=1, inplace=True)
analysis.drop('id', axis=1, inplace=True)
analysis.drop('click', axis=1, inplace=True)

In [8]:
analysis.head()

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,...,C14,C15,C16,C17,C18,C19,C20,C21,hour,day_of_week
0,1005,0,85f751fd,c4e18dd6,50e219e0,396df801,2347f47a,0f2161f8,a99f214a,554d9f5f,...,15705,320,50,1722,0,35,100084,79,0,Tuesday
1,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,79e904d3,...,15702,320,50,1722,0,35,-1,79,0,Tuesday
2,1005,0,85f751fd,c4e18dd6,50e219e0,6fc85e22,7801e8d9,0f2161f8,0dcbbf32,20dc99fb,...,18993,320,50,2161,0,35,-1,157,0,Tuesday
3,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,2c2f7780,...,15701,320,50,1722,0,35,100084,79,0,Tuesday
4,1005,1,d9750ee7,98572c79,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,f416abc8,...,17614,320,50,1993,2,1063,100084,33,0,Tuesday


In [9]:
def convert_obj_to_int(self):
    
    object_list_columns = self.columns
    object_list_dtypes = self.dtypes
    new_col_suffix = '_int'
    for index in range(0,len(object_list_columns)):
        if object_list_dtypes[index] == object :
            self[object_list_columns[index]+new_col_suffix] = self[object_list_columns[index]].map( lambda  x: hash(x))
            self.drop([object_list_columns[index]],inplace=True,axis=1)
    return self
analysis = convert_obj_to_int(analysis)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(analysis, target, test_size = 0.2, random_state =0 )

In [11]:
X_train.head()

,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,...,site_id_int,site_domain_int,site_category_int,app_id_int,app_domain_int,app_category_int,device_id_int,device_ip_int,device_model_int,day_of_week_int
10382,1005,0,1,2,19251,320,50,2201,3,35,...,8629950133425947508,111604357524640377,-2414393451864678829,-5019445241186748898,133654332410032911,4844916605846005300,-3518045965299665547,6576036695264281503,-662998938856388506,7099394789598017869
73171,1005,0,1,2,377,320,50,112,3,1319,...,8629950133425947508,111604357524640377,-2414393451864678829,8467682082009759696,-6327619399419000028,-1810359225251949021,-3518045965299665547,-7033288248019636936,-3203218385020632329,-4227466896491634454
30938,1005,0,1,0,15701,320,50,1722,0,35,...,1586783940683276736,-6461364947929047160,-6619379710939434495,-3459646385640828756,8511753678100510838,-3690605714457023166,-3518045965299665547,6969805819945366459,7568039529498303106,8580448175344127886
99310,1005,0,1,0,22104,320,50,2545,0,431,...,1586783940683276736,-6461364947929047160,-6619379710939434495,-3459646385640828756,8511753678100510838,-3690605714457023166,-3518045965299665547,7976586324066454687,8597697701124467158,8580448175344127886
58959,1005,0,1,0,20215,320,50,2316,0,167,...,-8391469780661372980,-6321313078051792942,-6827508670936997654,-3459646385640828756,8511753678100510838,-3690605714457023166,-3518045965299665547,4424232895115489706,7119642937421895456,-1539871469094565082


In [12]:
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

clf = tree.DecisionTreeClassifier(max_depth = 5000)
clf1 = clf.fit(X_train, y_train)
clf1_pred=clf1.predict(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,clf1_pred)


array([[13836,  2773],
       [ 2402,   989]])

In [13]:
pd.crosstab(y_test,clf1_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,13836,2773,16609
1,2402,989,3391
All,16238,3762,20000


In [14]:
from sklearn.metrics import log_loss

log_loss(clf1_pred,y_test)

8.937004423834505

In [15]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 10 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

In [16]:
# Train the model on training data
rf.fit(X_train, y_train);

# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [24]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [17]:
scores = cross_val_score(rf, X_train, y_train)
scores.mean() 

/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


0.8231875032260364

In [25]:
log_loss(predictions,y_test)

6.394106397998901

In [27]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())


Parameters currently in use:

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 'warn',
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 3)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


In [30]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 10 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=3, random_state=42)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=700, max_features=sqrt, min_samples_split=50, max_depth=70 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=700, max_features=sqrt, min_samples_split=50, max_depth=70, score=0.832937708828, total=12.8min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=700, max_features=sqrt, min_samples_split=50, max_depth=70 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 14.4min remaining:    0.0s


[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=700, max_features=sqrt, min_samples_split=50, max_depth=70, score=0.832918958851, total=13.4min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=700, max_features=sqrt, min_samples_split=50, max_depth=70 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 29.5min remaining:    0.0s


[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=700, max_features=sqrt, min_samples_split=50, max_depth=70, score=0.833000832502, total=13.4min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=100, max_features=auto, min_samples_split=10, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=100, max_features=auto, min_samples_split=10, max_depth=30, score=0.828253964683, total= 2.0min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=100, max_features=auto, min_samples_split=10, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=100, max_features=auto, min_samples_split=10, max_depth=30, score=0.827942715072, total= 2.0min
[CV] bootstrap=False, min_samples_leaf=1, n_estimators=100, max_features=auto, min_samples_split=10, max_depth=30 
[CV]  bootstrap=False, min_samples_leaf=1, n_estimators=100, max_features=auto, min_samples_split=10, max_depth=30, score=0.82845582114, total= 2.0min
[CV] bootstrap=False, min_samples_leaf=1, n_esti

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 272.8min finished


KeyboardInterrupt: 

In [31]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 50,
 'n_estimators': 700}